# Creating a ranking of goalies from the three models, expected_end_cost, expected_total_points, model_goalie_score

In [47]:
import pandas as pd
import numpy as np
import requests
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler


In [48]:
#Import goalie-specific, model ready csv that alrerady has the Saves_percentages and expected total points from the earlier 2 models - this is the master.

final_prediction_df_master = pd.read_csv("C:/Users/Daniel Quinn/Desktop/Bootcamp/Project_2/data/processed/final_prediction_df_master.csv")
#Ensure we can read the whole dataframe, without "..."
pd.set_option("display.max_rows", None)

final_prediction_df_master.columns

Index(['Unnamed: 0', 'id', 'first_name', 'second_name', 'team', 'element_type',
       'code', 'element_code', 'season', 'minutes', 'goals_scored', 'assists',
       'clean_sheets', 'goals_conceded', 'own_goals', 'penalties_saved',
       'penalties_missed', 'yellow_cards', 'red_cards', 'saves', 'bonus',
       'bps', 'influence', 'creativity', 'threat', 'ict_index', 'starts',
       'expected_goals', 'expected_assists', 'expected_goal_involvements',
       'expected_goals_conceded', 'Saves_Percentage', 'start_cost', 'end_cost',
       'Predicted_End_Cost', 'total_points', 'Predicted_Total_Points',
       'Goalie_Score', 'Predicted_Goalie_Score', 'Composite Score',
       'Final_Point_Ranking'],
      dtype='object')

In [50]:
#Import team name csv & merge so we have team name

team_name_list_and_id = pd.read_csv("C:/Users/Daniel Quinn/Desktop/Bootcamp/Project_2/data/processed/team_name_list_and_id.csv")
clean_team_name = team_name_list_and_id[['id', 'name', 'short_name']]
clean_team_name_2 = clean_team_name.rename(columns=
                                           {'id': 'team', 'name': 'team_name'})
clean_team_name_2 = clean_team_name_2[['team', 'team_name']]
clean_team_name_2

final_ranking = pd.merge(clean_team_name_2, final_prediction_df_master, on="team", how="inner")
final_ranking_dropna = final_ranking.dropna()
final_ranking_dropna.drop(columns=['Unnamed: 0'])
final_goalie_rank = final_ranking_dropna.sort_values('Final_Point_Ranking', ascending=False)
final_goalie_rank.columns

final_goalie_rank = final_goalie_rank[
    ['id', 'first_name', 'second_name',
       'element_type', 'code', 'element_code', 'Saves_Percentage', 'Predicted_End_Cost',
       'Predicted_Total_Points', 'Goalie_Score',
       'Predicted_Goalie_Score', 'Composite Score', 'Final_Point_Ranking']]

final_goalie_rank


,id,first_name,second_name,element_type,code,element_code,Saves_Percentage,Predicted_End_Cost,Predicted_Total_Points,Goalie_Score,Predicted_Goalie_Score,Composite Score,Final_Point_Ranking
105,383,André,Onana,1,202641,202641,71.980676,49.242,133.922,129.709179,128.524589,311.688589,311.688589
81,310,Alisson,Ramses Becker,1,116535,116535,73.451327,53.048,106.334,98.498230,97.583492,256.965492,306.728291
180,554,José,Malheiro de Sá,1,149065,149065,69.791667,48.082,109.312,111.387500,111.538895,268.932895,294.709926
119,413,Nick,Pope,1,98747,98747,72.881356,49.888,58.060,63.188136,62.894259,170.842259,290.391385
97,347,Ederson,Santana de Moraes,1,121160,121160,67.857143,54.652,124.784,78.714286,80.866004,260.302004,287.885394
2,15,David,Raya Martin,1,154561,154561,66.197183,54.892,135.108,66.263380,65.995358,255.995358,274.716985
151,508,Guglielmo,Vicario,1,184254,184254,64.327485,50.906,118.964,102.538012,104.036279,273.906279,273.906279
25,91,Mark,Flekken,1,118342,118342,64.606742,47.252,120.672,100.269663,101.018287,268.942287,268.942287
51,235,Jordan,Pickford,1,111234,111234,70.348837,50.334,157.568,120.507558,122.490993,330.392993,256.188263
70,248,Bernd,Leno,1,80201,80201,68.717949,48.616,137.162,116.614359,116.866484,302.644484,248.702722
